<div style="text-align: center;">
  <img src="https://github.com/Hack-io-Data/Imagenes/blob/main/01-LogosHackio/logo_amarillo@4x.png?raw=true" alt="esquema" />
</div>


# Laboratorio ETL: Análisis del Sistema Energético en España

## Objetivo

Durante todos los laboratorios de esta semana realizarás un proceso completo de ETL para analizar la relación entre la demanda, el consumo y la generación eléctrica en diferentes provincias de España a lo largo de un año. Además, complementarán este análisis con datos demográficos y económicos extraídos del Instituto Nacional de Estadística (INE). El **objetivo principal** del análisis es **examinar cómo la demanda, el consumo y la generación eléctrica en diferentes provincias de España a lo largo de los años están influenciados por factores demográficos y económicos, como la población y el PIB provincial**. El análisis busca identificar patrones y correlaciones entre estas variables para comprender mejor las dinámicas energéticas regionales y su relación con el desarrollo socioeconómico en España.

Antes de realizar el análisis, vamos a definir las hipótesis con las que vamos a trabajar, las cuales definirán todo tu análisis y planteamiento de los laboratorios: 

- **Hipótesis 1: La demanda eléctrica está correlacionada con la población de la provincia.** Provincias con mayor población tienden a tener una mayor demanda eléctrica.
  
- **Hipótesis 2: El crecimiento económico (medido por el PIB) está correlacionado con el consumo eléctrico.** Las provincias con un PIB más alto o en crecimiento experimentan un mayor consumo de energía.

- **Hipótesis 3: La proporción de generación renovable está relacionada con factores económicos o geográficos.** Provincias con un mayor desarrollo económico o con condiciones geográficas favorables (como más horas de sol o viento) tienden a generar más energía renovable.


## Tareas Laboratorio Transformación

En este laboratorio, tu objetivo será limpiar y preparar los datos extraídos previamente de diferentes fuentes para su posterior análisis. Trabajarás con datos provenientes de la API de Red Eléctrica Española (REE) y del Instituto Nacional de Estadística (INE). Estos datos incluyen información sobre demanda y generación eléctrica a nivel provincial, así como datos demográficos y económicos. 


- Cargar los Datos Extraídos:

  - **Demanda Eléctrica:** Carga los datos de demanda eléctrica extraídos de la API de REE.

  - **Generación Eléctrica:** Carga los datos de generación eléctrica diferenciados por tipo de energía (eólica, solar, hidroeléctrica, etc.) a nivel provincial.

  - **Datos Demográficos:** Carga los datos demográficos por provincia extraídos de la web del INE.

  - **Datos Económicos:** Carga los datos del PIB por provincia obtenidos del INE.


-Limpieza de Datos:

- Datos de la API de REE:

  - **Demanda Eléctrica:**

    - **Conversión de Timestamps:** Asegúrate de que las fechas estén correctamente formateadas en `datetime`. Si es necesario, convierte los datos a un formato uniforme (por ejemplo, `YYYY-MM` para datos mensuales).

    - **Tratamiento de Valores Nulos:** Identifica y maneja los valores nulos en caso de que los haya. Puedes optar por eliminar filas con valores faltantes.

    - **Estandarización de Nombres de Provincias:** Verifica que los nombres de las provincias estén estandarizados y coincidan en todos los conjuntos de datos. Si hay inconsistencias, corrígelas.

  - **Generación Eléctrica:**

    - **Desagregación de Tecnologías:** Asegúrate de que los datos estén correctamente desglosados por tipo de energía. Revisa que los campos correspondientes a energía eólica, solar, hidroeléctrica, etc., estén bien identificados y sin errores.

    - **Normalización de Unidades:** Verifica que las unidades de energía estén estandarizadas (por ejemplo, MWh). Realiza las conversiones necesarias si se encuentran en otras unidades.

    - **Identificación de Outliers:** Revisa los valores extremos o atípicos en la generación de energía y decide si deben ser tratados o eliminados.

- Datos del INE:

  - **Datos Demográficos:**

    - **Consistencia en la Codificación de Provincias:** Asegúrate de que los nombres de las provincias en los datos demográficos coincidan con los nombres utilizados en los datos eléctricos.

    - **Revisión de Categorías:** Verifica que las categorías de edad, sexo, y nacionalidad estén correctamente etiquetadas y sean consistentes en todo el dataset.

    - **Manejo de Valores Faltantes:** Revisa la presencia de valores faltantes y decide cómo tratarlos (relleno, eliminación o sustitución).

  - **Datos Económicos:**

    - **Normalización del PIB:** Si los datos del PIB están en diferentes unidades o escalas, asegúrate de normalizarlos para que sean comparables entre provincias.

    - **Agrupación Temporal:** Si los datos económicos están disponibles en diferentes periodos temporales, agrúpalos y normalízalos para que coincidan con los datos eléctricos en términos de granularidad temporal (mensual o anual).

NOTA: Ten en cuenta que los datos los vamos a tener que insertar en una base de datos mañana, por lo que toda esta limpieza os recomendamos que la penséis para poder crear e insertar los datos mañana. 

In [213]:
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt

In [214]:
def get_outliers(data):
    """
    Detecta valores atípicos en un conjunto de datos utilizando el método del rango intercuartílico (IQR).

    Args:
        data (Series): Serie de pandas que contiene los datos numéricos para analizar.

    Returns:
        Series: Serie de pandas que contiene los valores atípicos detectados.
    """
    q1,q3 = data.quantile([0.25,0.75])
    iqr = q3-q1
    upper_fence = q3 + (1.5*iqr)
    lower_fence = q1 - (1.5*iqr)
    outliers = data[(data < lower_fence) | (data > upper_fence)]
    # print(iqr)
    # print(lower_fence, upper_fence)
    return outliers

def decimalpoint(string_num):
    return float(string_num.replace('.', '').replace(',','.'))

In [215]:
cod_comunidades = {'Ceuta': 8744,
                    'Melilla': 8745,
                    'Andalucía': 4,
                    'Aragón': 5,
                    'Cantabria': 6,
                    'Castilla - La Mancha': 7,
                    'Castilla y León': 8,
                    'Cataluña': 9,
                    'País Vasco': 10,
                    'Principado de Asturias': 11,
                    'Comunidad de Madrid': 13,
                    'Comunidad Foral de Navarra': 14,
                    'Comunitat Valenciana': 15,
                    'Extremadura': 16,
                    'Galicia': 17,
                    'Illes Balears': 8743,
                    'Canarias': 8742,
                    'Región de Murcia': 21,
                    'La Rioja': 20}

code_mapper = dict()
for k,v in cod_comunidades.items():
    code_mapper[v] = k

## Demanda y generación energética

In [216]:
def extraer_df(ruta):
    years = os.listdir(ruta)
    df_final = pd.DataFrame()
    for year in years:
        ruta_year = os.path.join(ruta, year)
        files = os.listdir(ruta_year)
        df_com = pd.DataFrame()
        for file in files:
            ruta_file = os.path.join(ruta_year, file)
            df_archivo = pd.read_csv(ruta_file, index_col=0)
            df_archivo["code_com"] = cod_comunidades[file.split('.')[0]]
            df_com = pd.concat([df_com, df_archivo])
        df_final = pd.concat([df_final,df_com])
        df_final.reset_index(drop=True, inplace=True)
    df_final["fecha"] = df_final["datetime"].apply(lambda x :pd.to_datetime(x).strftime('%Y-%m'))
    df_final.drop(columns='datetime', inplace=True)
    return df_final

In [217]:
df_demanda = extraer_df(ruta='datos\demanda')
df_generacion = extraer_df(ruta='datos\generacion')

Vemos que la columna de magnitude no nos da información relevante.

In [218]:
df_generacion.sample(5)

,value,percentage,type,magnitude,code_com,fecha
647,2.485544e+04,0.084075,Solar fotovoltaica,NaN,15,2019-12
2671,1.794168e+05,0.152856,Solar fotovoltaica,NaN,7,2021-02
2454,1.308594e+06,0.820323,Eólica,NaN,5,2021-01
1050,8.436462e+04,1.000000,Generación renovable,NaN,10,2019-10
2827,1.220688e+04,0.021763,Otras renovables,NaN,9,2021-02


Está entera vacía

In [219]:
df_generacion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3557 entries, 0 to 3556
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   value       3557 non-null   float64
 1   percentage  3557 non-null   float64
 2   type        3557 non-null   object 
 3   magnitude   0 non-null      float64
 4   code_com    3557 non-null   int64  
 5   fecha       3557 non-null   object 
dtypes: float64(3), int64(1), object(2)
memory usage: 166.9+ KB


In [220]:
df_generacion.drop(columns='magnitude', inplace=True)

Podemos comprobar en la web de la [REE](https://www.ree.es/es/datos/generacion/estructura-generacion), que los datos energéticos están recogidos en MWh.

In [221]:
df_generacion.rename(columns={"value" : "energia (MWh)"}, inplace=True)

In [222]:
df_generacion.sample(10)

,energia (MWh),percentage,type,code_com,fecha
2305,11436.247,0.047272,Hidráulica,21,2020-08
334,23358.473,0.014471,Otras renovables,7,2019-11
516,13467.019,0.402100,Otras renovables,13,2019-01
2994,377426.174,0.865221,Eólica,15,2021-01
109,5342.305,0.009388,Otras renovables,5,2019-02
1436,6753.154,0.149717,Otras renovables,6,2020-12
1409,4524.090,0.215196,Eólica,6,2020-09
2495,1085380.367,1.000000,Generación renovable,5,2021-06
2899,7136.069,0.110165,Residuos renovables,13,2021-02
921,653.245,0.003597,Otras renovables,20,2019-01


Vamos a ver dónde encontramos los outliers:

In [223]:
df_generacion.iloc[get_outliers(df_generacion["energia (MWh)"]).index]['code_com'].map(code_mapper).value_counts()

code_com
Castilla y León               96
Galicia                       91
Andalucía                     71
Castilla - La Mancha          68
Aragón                        64
Cataluña                      39
Extremadura                   27
Principado de Asturias         6
Comunitat Valenciana           4
Comunidad Foral de Navarra     3
Name: count, dtype: int64

Sobretodo en Castilla y León y Galicia. En cuanto a las fechas donde se concentran estos valores atípicos:

In [224]:
df_generacion.iloc[get_outliers(df_generacion["energia (MWh)"]).index]['fecha'].value_counts().head()

fecha
2021-01    17
2021-07    17
2021-06    16
2020-05    16
2019-11    15
Name: count, dtype: int64

## INE: info demográfica y PIB

In [225]:
df_demog = pd.read_csv('datos\INE\datos_demograficos.csv', encoding='latin-1', sep=';')
provincias = df_demog["Provincias"].unique()

Primero voy a hacer un diccionario para mapear las diferentes provincias a los códigos.

In [226]:
import re

cods = dict()
for provincia in provincias:
    splitted = provincia.split()
    cods["".join(re.findall(pattern= r" (\D+)", string=provincia))] = splitted[0]

cod_provincias = dict()
for k,v in cods.items():
    try:
        cod_provincias[k] = int(v)
    except:
        cod_provincias[k] = v

In [227]:
df_demog["Provincias"] = df_demog["Provincias"].apply(lambda x:"".join(re.findall(pattern= r" (\D+)", string=x))).map(cod_provincias)

In [228]:
df_demog["Total"] = df_demog["Total"].apply(decimalpoint)

In [229]:
df_demog.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7632 entries, 0 to 7631
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Provincias               7632 non-null   object 
 1   Edad (3 grupos de edad)  7632 non-null   object 
 2   Españoles/Extranjeros    7632 non-null   object 
 3   Sexo                     7632 non-null   object 
 4   Año                      7632 non-null   int64  
 5   Total                    7632 non-null   float64
dtypes: float64(1), int64(1), object(4)
memory usage: 357.9+ KB


In [284]:
df_demog

,Provincias,Edad (3 grupos de edad),Españoles/Extranjeros,Sexo,Año,Total
0,TOTAL,TOTAL EDADES,TOTAL,Ambos sexos,2021,47385107.00
1,TOTAL,TOTAL EDADES,TOTAL,Ambos sexos,2020,47450795.00
2,TOTAL,TOTAL EDADES,TOTAL,Ambos sexos,2019,47026208.00
3,TOTAL,TOTAL EDADES,TOTAL,Hombres,2021,23222953.00
4,TOTAL,TOTAL EDADES,TOTAL,Hombres,2020,23255590.00
...,...,...,...,...,...,...
7627,52,65 y más,% Extranjeros,Hombres,2020,10.86
7628,52,65 y más,% Extranjeros,Hombres,2019,10.73
7629,52,65 y más,% Extranjeros,Mujeres,2021,9.55
7630,52,65 y más,% Extranjeros,Mujeres,2020,9.55


In [267]:
df_pib = pd.read_csv('datos\INE\datos_pib.csv', encoding='latin-1', sep = ';')

In [268]:
df_pib["Provincias"] = df_pib["Provincias"].apply(lambda x:"".join(re.findall(pattern= r" (\D+)", string=x))).map(cod_provincias)

In [269]:
df_pib["periodo"] = df_pib["periodo"].str.replace('(P)', "").apply(int)

In [270]:
df_pib["Total"] = df_pib["Total"].apply(decimalpoint)

In [272]:
df_pib.drop(index= df_pib[df_pib["Ramas de actividad"] == 'PRODUCTO INTERIOR BRUTO A PRECIOS DE MERCADO'].index, inplace=True)

In [274]:
df_pib.reset_index(drop=True)

,Provincias,Ramas de actividad,periodo,Total
0,2,"A. Agricultura, ganadería, silvicultura y pesca",2021,884324.0
1,2,"A. Agricultura, ganadería, silvicultura y pesca",2020,768976.0
2,2,"A. Agricultura, ganadería, silvicultura y pesca",2019,791464.0
3,2,"B_E. Industrias extractivas, industria manufac...",2021,1397006.0
4,2,"B_E. Industrias extractivas, industria manufac...",2020,1194438.0
...,...,...,...,...
1399,52,Valor añadido bruto total,2020,1381519.0
1400,52,Valor añadido bruto total,2019,1473873.0
1401,52,Impuestos netos sobre los productos,2021,153994.0
1402,52,Impuestos netos sobre los productos,2020,132490.0


In [254]:
df_generacion.groupby(["fecha", "code_com", "type", "percentage"])["energia (MWh)"].mean().reset_index()

,fecha,code_com,type,percentage,energia (MWh)
0,2019-01,4,Eólica,0.638703,547209.8430
1,2019-01,4,Generación renovable,1.000000,856751.4350
2,2019-01,4,Hidráulica,0.029517,25288.6050
3,2019-01,4,Otras renovables,0.127868,109551.1710
4,2019-01,4,Solar fotovoltaica,0.116924,100175.1370
...,...,...,...,...,...
3552,2021-12,8743,Residuos renovables,0.435657,8866.0930
3553,2021-12,8743,Solar fotovoltaica,0.548044,11153.2820
3554,2021-12,8745,Generación renovable,1.000000,540.1755
3555,2021-12,8745,Residuos renovables,1.000000,536.5155


In [280]:
df_generacion[(df_generacion["percentage"] == 1) & (df_generacion["type"] == 'Solar fotovoltaica')]

,energia (MWh),percentage,type,code_com,fecha
3328,3.66,1.0,Solar fotovoltaica,8745,2021-12


In [283]:
df_generacion[(df_generacion["code_com"] == 8745)&(df_generacion["fecha"] == '2021-12')]

,energia (MWh),percentage,type,code_com,fecha
3328,3.6600,1.0,Solar fotovoltaica,8745,2021-12
3340,536.5155,1.0,Residuos renovables,8745,2021-12
3351,540.1755,1.0,Generación renovable,8745,2021-12


In [298]:
df_generacion[(df_generacion["fecha"] == '2021-12')&(df_generacion["code_com"] == 13)]

,energia (MWh),percentage,type,code_com,fecha
2873,4046.7560,0.133001,Hidráulica,13,2021-12
2885,3622.2120,0.119048,Solar fotovoltaica,13,2021-12
2897,15386.2810,0.505686,Otras renovables,13,2021-12
2909,7371.3225,0.242266,Residuos renovables,13,2021-12
2921,30426.5715,1.000000,Generación renovable,13,2021-12


In [303]:
df_demanda["fecha"] = pd.to_datetime(df_demanda["fecha"])

In [308]:
df_demanda["mes"] = df_demanda["fecha"].apply(lambda x: x.month)
df_demanda["year"] = df_demanda["fecha"].apply(lambda x: x.year)

In [312]:
df_demanda.drop(columns="fecha", inplace=True)

In [315]:
df_demanda.groupby('year')["value"].mean()

year
2019    1.160806e+06
2020    1.096713e+06
2021    1.125201e+06
Name: value, dtype: float64

In [281]:
cod_comunidades

{'Ceuta': 8744,
 'Melilla': 8745,
 'Andalucía': 4,
 'Aragón': 5,
 'Cantabria': 6,
 'Castilla - La Mancha': 7,
 'Castilla y León': 8,
 'Cataluña': 9,
 'País Vasco': 10,
 'Principado de Asturias': 11,
 'Comunidad de Madrid': 13,
 'Comunidad Foral de Navarra': 14,
 'Comunitat Valenciana': 15,
 'Extremadura': 16,
 'Galicia': 17,
 'Illes Balears': 8743,
 'Canarias': 8742,
 'Región de Murcia': 21,
 'La Rioja': 20}

In [253]:
df_generacion.fecha.unique()

array(['2019-01', '2019-02', '2019-03', '2019-04', '2019-05', '2019-06',
       '2019-07', '2019-08', '2019-09', '2019-10', '2019-11', '2019-12',
       '2020-01', '2020-02', '2020-03', '2020-04', '2020-05', '2020-06',
       '2020-07', '2020-08', '2020-09', '2020-10', '2020-11', '2020-12',
       '2021-01', '2021-02', '2021-03', '2021-04', '2021-05', '2021-06',
       '2021-07', '2021-08', '2021-09', '2021-10', '2021-11', '2021-12'],
      dtype=object)